In [40]:
#A3

# 画像とテキストの両方を入力として渡す
import base64
import requests
import os
from openai import OpenAI
import openai
import json

# OpenAI API Key
api_key = os.getenv('OPENAI_API_KEY_NEW')

# 関数：画像をエンコードする
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 画像のパス
image_path = "20240118081021110.jpg"

# 画像をbase64にエンコード
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# JSONファイルを読み込む
with open('20240118081021110_0.json', 'r', encoding='utf-8') as file:
    prompt = json.load(file)

payload = {
    "model": "gpt-4-turbo",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"{prompt}"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        },
        {"role": "system", "content": 
"""

## 依頼するタスク ##
１．所定の「項目」が存在する場合に限りそれを抽出してください。
２．それぞれの項目について、抽出結果の信頼性（％）を評価して下さい。
３．抽出した項目は、所定の「出力形式」に従って整形してください。
４．読み取った一番上の行の形式を踏襲して、それ以降下の行も読み取っていってください。


## 抽出する項目 ##
発注番号
F
紙質
巾
流れ
丁取り
罫線
枚数
納期
備考

## 出力形式 ##
- 項目の抽出結果はcsv形式で出力してください。
- 表の足りない部分は"不明"で埋めて下さい。

## 指示とアドバイス ##
- ステップバイステップで考えてください。
- 深呼吸して考えてください。
- この分析は私のキャリアにとってとても重要です。
- 自分の能力を信じて、もっと上を目指してください。
- このタスクにおいて、最高の結果を出すために、追加の情報が必要な場合は、逆質問をしてください。
- 不明確な項目は"不明"と出力すること
- 数値は整数で出力すること。
- 整数に小数点がついてる場合は切り捨てること。
- 罫線の列は例えば、"158/145/158"もしくは"(158+145+158=461)"のようになっていることが多いです。これを認識してください。
- 丁取りは項目としてない場合も多いです。
- 丁取りは1桁の整数であることに注意してください。
- 出力するjsonデータが複数ある場合は信憑性の最も高いものを1つだけ出力して下さい。
- "スリッタ寸法"というカラムがある場合、それは"罫線"と解釈してください。
- "F"と"フルート"は同じと解釈してください。
- "F"または"フルート"のカラムは"A"か"B"か"W"の３つのいずれかでしかありません。したがってそれ以外のものを認識しようとしている場合それは間違いです。
- "KSSK5"や"KSK5"や"C5SC5"などは"紙質"を意味します。
- "納期"は例えば"1/19"などの形式で書かれていることが多いです。
- "枚数"と"数量"は同じ意味と解釈して、"枚数"として出力して下さい。
- "1250x1240"は基本"巾x流れ"を表しています。この場合は巾:1250、流れ:1240のようにしてください。
- 抽出する項目は順不同である。最も妥当な項目に内容が対応するように読み取ってください。
- 入力の画像に例えば"No."の項目がない場合これは出力しないものとする。
- このタスクにおいて、最高の結果を出すために、追加の情報が必要な場合は、逆質問をしてください。
- 複数行ある場合は1行1行注意してすべての行を出力してください。
""" 

    
    }

        
    ],
    "max_tokens": 1200
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9HW8kUZbda2y9uOwqR6PYd3Tnkp8N', 'object': 'chat.completion', 'created': 1713962150, 'model': 'gpt-4-turbo-2024-04-09', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '以下の情報をCSV形式で出力します：\n\n```\n発注番号,F,紙質,巾,流れ,丁取り,罫線,枚数,納期,備考\n250242,W,K5SK5,1250,1250,不明,不明,17,1/19,不明\n250243,A,K5K5,1250,1240,不明,不明,17,1/19,不明\n250244,A,K5K5,1200,1550,不明,(124+340+124)=588,101,1/19,不明\n250245,A,K5K5,1300,1772,不明,不明,12,1/19,不明\n250246,A,K5K5,1350,995,不明,不明,21,1/19,不明\n250248,W,C5SC5,1250,1507,不明,不明,16,1/19,不明\n```\n\nこの出力において、紙質を英字の種類で、枚数を整数で、納期を特定のフォーマットで正確に抽出しています。また、不明な情報は"不明"と記載してあります。提供された情報と照らし合わせた結果、これが最も信頼性の高い出力だと判断します。'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 15587, 'completion_tokens': 349, 'total_tokens': 15936}, 'system_fingerprint': 'fp_67e6987839'}


In [42]:
import pandas as pd

# 抽出された情報をCSV形式で定義
csv_data = """
発注番号,F,紙質,巾,流れ,丁取り,罫線,枚数,納期,備考
250242,W,K5SK5,1250,1250,不明,不明,17,1/19,不明
250243,A,K5K5,1250,1240,不明,不明,17,1/19,不明
250244,A,K5K5,1200,1550,不明,(124+340+124)=588,101,1/19,不明
250245,A,K5K5,1300,1772,不明,不明,12,1/19,不明
250246,A,K5K5,1350,995,不明,不明,21,1/19,不明
250248,W,C5SC5,1250,1507,不明,不明,16,1/19,不明
"""

# CSVデータをPandasのDataFrameに変換
df = pd.read_csv(pd.compat.StringIO(csv_data))

# データの表示
df

AttributeError: module 'pandas.compat' has no attribute 'StringIO'

In [39]:
import pandas as pd

# 抽出された情報をCSV形式で定義
csv_data = """
発注番号,F,紙質,巾,流れ,丁取り,罫線,枚数,納期,備考
P0001/0001,A,C5XC5,950,830,不明,(158+145+158=461),10,23.03.01,#59 用パット
"""

# CSVデータをPandasのDataFrameに変換
df = pd.read_csv(pd.compat.StringIO(csv_data))

# データの表示
print(df)

AttributeError: module 'pandas.compat' has no attribute 'StringIO'